In [54]:
import os
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict



In [55]:
# path_P658207LE0A_SEGMENT_Model = "./../models/P658207LE0A_SEGMENT.pt"
# P658207LE0A_SEGMENT_Model = YOLO(path_P658207LE0A_SEGMENT_Model)

path_CLIP_Model = "./../models/P8462284S00_detect.pt"
path_SEGMENT_Model = "./../models/P8462284S00_segment.pt"
CLIP_Model = YOLO(path_CLIP_Model)
SEGMENT_Model = YOLO(path_SEGMENT_Model)


In [56]:
SAHI_CLIP_Model = AutoDetectionModel.from_pretrained(model_type="yolov8",model_path=path_CLIP_Model,
                                                                    confidence_threshold=0.5,   
                                                                    device="cuda:0")

In [57]:
#get current working directory
cwd = os.getcwd()
print(cwd)

c:\Users\AIKENSA 05\Documents\aikensa_dcam_gaikan_v0.1.1\aikensa\tools


In [58]:
image_path = "./../tools/sample/01.png"

image = cv2.imread(image_path)
#rgb to bgr
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [59]:
ClipDetection = get_sliced_prediction(
            image, 
            SAHI_CLIP_Model,
            slice_height=497, slice_width=1960, 
            overlap_height_ratio=0.0, overlap_width_ratio=0.2,
            postprocess_match_metric="IOS",
            postprocess_match_threshold=0.005,
            postprocess_class_agnostic=True,
            postprocess_type="GREEDYNMM",
            verbose=0,
            perform_standard_pred=False
        )

ClipDetection.export_visuals(export_dir="demo_data/", rect_th=1, hide_labels=True, hide_conf=True)


In [60]:
result = CLIP_Model(image_path, imgsz=1960, save=True, visualize=False, show_boxes=False, conf=0.01)



WARNING  imgsz=[1960] must be multiple of max stride 32, updating to [1984]
image 1/1 c:\Users\AIKENSA 05\Documents\aikensa_dcam_gaikan_v0.1.1\aikensa\tools\..\tools\sample\01.png: 192x1984 11 CLIPs, 11.5ms
Speed: 2.0ms preprocess, 11.5ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 1984)
Results saved to runs\detect\predict12


In [61]:
#predict segmentations
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image1 = image[:, :1640, :]
image2 = image[:, -1640:, :]
segmentation = SEGMENT_Model(image1, imgsz=1280, save=True, visualize=False, show_boxes=False, conf=0.3, retina_masks=True)
segmentation = SEGMENT_Model(image2, imgsz=1280, save=True, visualize=False, show_boxes=False, conf=0.3, retina_masks=True)


0: 416x1280 1 WS, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 1280)
Results saved to runs\segment\predict

0: 416x1280 1 WS, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 1280)
Results saved to runs\segment\predict
